<a href="https://colab.research.google.com/github/protontypes/osta/blob/main/packages_insights.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install pyalex semanticscholar habanero itables

In [2]:
import requests
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import pyalex
pyalex.config.email = "ly0@protonmail.com"
from habanero import counts

from semanticscholar import SemanticScholar
sch = SemanticScholar()

URL = "https://ost.ecosyste.ms/api/v1/projects/packages"
FILE_TO_SAVE_AS = "ecosystems_downloads.json" # the name you want to save file as

import os.path
#os.path.isfile(fname)

resp = requests.get(URL) # making requests to server

with open(FILE_TO_SAVE_AS, "wb") as f: # opening a file handler to create new file
    f.write(resp.content) # writing content to file

In [4]:
import pandas as pd

# Define the S3 path to the Parquet file
s3_path = 's3://anaconda-package-data/conda/monthly/2024/2024-12.parquet'

# Load the Parquet file into a DataFrame
df = pd.read_parquet(s3_path, engine='pyarrow')
package_name = 'pypsa'
package_data = df[df['pkg_name'] == package_name]
total_downloads = package_data['counts'].sum()
print(f'Total downloads for {package_name}: {total_downloads}')

Total downloads for pypsa: 4126


In [ ]:
df = pd.read_json(resp.content.decode())

In [ ]:
df.head(30)

In [ ]:
df.iloc[1]["packages"][0] # GO packages do not provide montly download numbers. That's why we had to kick out kube-green

In [ ]:
names = []
download_counts = []
url = []
description = []
category = []
sub_category = []
language = []
docker_downloads = []
doi = []
citations = []

for index, row in df.iterrows():
    names.append(row['name'])
    package_downloads = 0
    docker_download_count = 0
    for package_manager in range(len(row['packages'])):
        if row['packages'][package_manager]['downloads']:
            if row['packages'][package_manager]['downloads_period'] == "last-month":
                package_downloads += row['packages'][package_manager]['downloads']
        if row['packages'][package_manager]['docker_downloads_count']:
            docker_download_count = row['packages'][package_manager]['docker_downloads_count']
        print(package_manager)
    download_counts.append(package_downloads)
    docker_downloads.append(docker_download_count)
    url.append(row['url'])
    description.append(row['description'])
    category.append(row['category'])
    sub_category.append(row['sub_category'])
    language.append(row['language'])
    citations.append(row['total_citations'])


In [ ]:
df_extract = pd.DataFrame()
df_extract['project_names'] = names
df_extract['download_counts'] = download_counts
df_extract['git_url'] = url
df_extract['description'] = description
df_extract['category'] = category
df_extract['sub_category'] = sub_category
df_extract['language'] = language
df_extract['docker_downloads'] = docker_downloads
df_extract['citations'] = citations
df_extract.head(200)

df_extract = df_extract[(df_extract['category'] == 'Renewable Energy') | (df_extract['category'] == 'Energy Systems') | (df_extract['category'] == 'Energy Storage')]


In [ ]:
df_extract.loc[df['language'] == 'Julia']

In [ ]:
import textwrap

def text_to_link(project_name, git_url):
    return '<a href="' + git_url + '" target="_blank" style = "color: black">' + str(project_name) + "</a>"

def text_to_bolt(topic):
    return "<b>" + topic + "</b>"

def line_break_text(text):
    wrapped_text = '<br>'.join(textwrap.wrap(text, 64))
    return wrapped_text

df_extract["description"] = df_extract.apply(
    lambda x: line_break_text(x.description), axis=1
)


df_extract["project_names"] = df_extract.apply(
    lambda x: text_to_link(x.project_names, x.git_url), axis=1
)
df_extract

In [ ]:
import numpy as np
import pandas as pd
import plotly.io as pio
import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime

# Get the current date
current_date = datetime.now()

# Format the date to display the full month and year
month_year = current_date.strftime("%B %Y")


#df_extract = df_extract[df_extract["category"] == "Renewable Energy"]

number_of_projects_to_show = 300
top_downloaders = df_extract.nlargest(number_of_projects_to_show, "download_counts")
top_downloaders.index.name = "ranking"
color_discrete_sequence = px.colors.qualitative.Vivid
fig = px.bar(
    top_downloaders,
    x=top_downloaders["download_counts"],
    y=top_downloaders["project_names"],
    custom_data=["project_names", "download_counts", "docker_downloads", "git_url", "description","category","sub_category","language",top_downloaders.index+1],
    orientation="h",
    color = 'category',
    color_discrete_sequence=color_discrete_sequence,
)

fig.update_layout(
    height=number_of_projects_to_show*20,  # Added parameter
    #width=700,
    xaxis_title="",
    yaxis_title=None,
    title="Package Downloads of Open Source Project in Climate and Sustainability in "+month_year,
    dragmode=False,
)

fig.update_layout(hovermode="y unified",hoverdistance=1000)

fig.update_traces(
    hovertemplate="<extra></extra>"+"<br>".join(
        [
            "Ranking: <b>%{customdata[8]}</b>",
            "Description: <b>%{customdata[4]}</b>",
            "Sub Category: <b>%{customdata[6]}</b>",
            "Language: <b>%{customdata[7]}</b>",
            "Downloads per month: <b>%{customdata[1]}</b>",
            #"Docker Downloads: <b>%{customdata[2]}</b>",
            "Category: <b>%{customdata[5]}</b>",

        ]
    )

)

#fig.update(layout_showlegend=False)

fig.update_layout(modebar=dict(bgcolor="rgba(0,0,0,0)"))

config = {
  'toImageButtonOptions': {
    'format': 'png', # one of png, svg, jpeg, webp
  },
  'responsive':'true'
}
fig.update_layout(
    plot_bgcolor='white'
)

fig.update_layout(modebar_color="#009485", modebar_activecolor="#2563eb")
fig["layout"]["yaxis"]["autorange"] = "reversed"

fig.update_xaxes(showspikes=False)  # <-- add this line
fig.update_yaxes(showspikes=False)  # <-- add this line
#fig.update_layout(paper_bgcolor = "rgba(0,0,0,0)",
#                  plot_bgcolor = "rgba(0,0,0,0)")

#fig.update_layout(barmode='stack', xaxis={'categoryorder':'total ascending'})
fig.update_layout(xaxis_type = "log",yaxis_categoryorder = 'total descending',legend_title=None, xaxis={'side': 'top'}, )
fig.show(config=config)

pio.write_json(fig,"download_plot.json")
pio.write_html(fig,"download_plot.html")

In [ ]:
from itables import init_notebook_mode

init_notebook_mode(all_interactive=True)
df_extract.loc[df_extract['citations'] > 0]